In [303]:
import os
import torch
import torchaudio
from transformers import AutoFeatureExtractor, ASTFeatureExtractor, ASTForAudioClassification, ASTModel

In [300]:
import librosa as aud
import soundfile as sf

import faiss

In [301]:
from IPython.display import Audio

In [2]:
feature_extractor = AutoFeatureExtractor.from_pretrained('MIT/ast-finetuned-audioset-10-10-0.4593')

In [176]:
model = ASTModel.from_pretrained('MIT/ast-finetuned-audioset-10-10-0.4593')

Some weights of the model checkpoint at MIT/ast-finetuned-audioset-10-10-0.4593 were not used when initializing ASTModel: ['classifier.dense.bias', 'classifier.layernorm.bias', 'classifier.layernorm.weight', 'classifier.dense.weight']
- This IS expected if you are initializing ASTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ASTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
y, sr = aud.load('/Users/raghothams/Downloads/Pink-Floyd-High-Hopes-Official-Music-Video-HD (1).wav')

In [13]:
y.shape

(661500,)

In [14]:
sr

22050

In [16]:
hh = aud.resample(y, orig_sr=sr, target_sr=16000)

In [17]:
hh

array([-0.00202585, -0.00331331, -0.00263065, ..., -0.02626844,
       -0.01396614, -0.01353868], dtype=float32)

In [19]:
sf.write('/Users/raghothams/Downloads/high-hopes.wav', hh, samplerate=16000)

In [22]:
features = feature_extractor(hh, sampling_rate=16000)

In [30]:
features['input_values'][0]

array([[-0.7224061 , -1.0962328 , -0.7194104 , ..., -0.45379865,
        -0.7194977 , -1.2775939 ],
       [-0.65820986, -1.0856003 , -0.7087779 , ..., -0.6819102 ,
        -0.7962905 , -1.2724096 ],
       [-0.80300397, -1.2611395 , -0.8843171 , ..., -0.671455  ,
        -1.0123867 , -1.2775939 ],
       ...,
       [-0.6379124 , -1.072842  , -0.69601953, ..., -0.63410664,
        -0.65513176, -0.98822194],
       [-0.68537515, -1.1313919 , -0.75456953, ..., -0.6059823 ,
        -0.6410481 , -0.8586102 ],
       [-0.73471755, -1.0880506 , -0.71122825, ..., -0.5132176 ,
        -0.60985035, -0.9817077 ]], dtype=float32)

In [31]:
from IPython.display import Audio

In [32]:
Audio(data=hh, rate=sr)

In [251]:
y, sr = aud.load('/Users/raghothams/Downloads/Anchor-Point.wav')

In [266]:
sr

22050

In [267]:
len(y)

7916489

In [268]:
num_secs = 10

In [270]:
for i in range(0, len(y),  num_secs * sr):
#     print(i)
    print(i,num_secs * sr + i)
    y_slice = y[i:num_secs * sr + i]
    y_slice = aud.resample(y_slice, orig_sr=sr, target_sr=16000)
    sf.write("./data/ap"+str(i)+".wav", y_slice, 16000)

0 220500
220500 441000
441000 661500
661500 882000
882000 1102500
1102500 1323000
1323000 1543500
1543500 1764000
1764000 1984500
1984500 2205000
2205000 2425500
2425500 2646000
2646000 2866500
2866500 3087000
3087000 3307500
3307500 3528000
3528000 3748500
3748500 3969000
3969000 4189500
4189500 4410000
4410000 4630500
4630500 4851000
4851000 5071500
5071500 5292000
5292000 5512500
5512500 5733000
5733000 5953500
5953500 6174000
6174000 6394500
6394500 6615000
6615000 6835500
6835500 7056000
7056000 7276500
7276500 7497000
7497000 7717500
7717500 7938000


In [271]:
!ls ./data

ap0.wav       ap441000.wav  ap7717500.wav hh22.wav      hh38.wav
ap1102500.wav ap4410000.wav ap882000.wav  hh23.wav      hh39.wav
ap1323000.wav ap4630500.wav hh0.wav       hh24.wav      hh4.wav
ap1543500.wav ap4851000.wav hh1.wav       hh25.wav      hh40.wav
ap1764000.wav ap5071500.wav hh10.wav      hh26.wav      hh41.wav
ap1984500.wav ap5292000.wav hh11.wav      hh27.wav      hh42.wav
ap220500.wav  ap5512500.wav hh12.wav      hh28.wav      hh43.wav
ap2205000.wav ap5733000.wav hh13.wav      hh29.wav      hh44.wav
ap2425500.wav ap5953500.wav hh14.wav      hh3.wav       hh45.wav
ap2646000.wav ap6174000.wav hh15.wav      hh30.wav      hh46.wav
ap2866500.wav ap6394500.wav hh16.wav      hh31.wav      hh5.wav
ap3087000.wav ap661500.wav  hh17.wav      hh32.wav      hh6.wav
ap3307500.wav ap6615000.wav hh18.wav      hh33.wav      hh7.wav
ap3528000.wav ap6835500.wav hh19.wav      hh34.wav      hh8.wav
ap3748500.wav ap7056000.wav hh2.wav       hh35.wav      hh9.wav
ap3969000.wav ap7276500.wav hh2

In [70]:
y[0:220500].shape

(220500,)

In [71]:
y[220500:441000].shape

(220500,)

In [72]:
y.shape

(10338817,)

In [275]:
d = 768
index = faiss.IndexFlatL2(d)

In [276]:
index_id = 0
faiss_index_lookup_samples = {}

In [234]:
o = model(**input_values)

In [235]:
o.last_hidden_state.shape

torch.Size([1, 1214, 768])

In [278]:
for file in os.listdir('./data/'):
    print(file)
    if 'ap' in file:
        print(f'./data/{file}')
        waveform, sample_rate = torchaudio.load(f'./data/{file}')
        waveform = waveform.squeeze().numpy()
        input_values = feature_extractor(waveform, return_tensors="pt", sampling_rate=sample_rate)
        with torch.no_grad():
            embeddings = model(**input_values).last_hidden_state
            print(embeddings.shape)
            embeddings = embeddings.mean(dim=1)
            print(embeddings.shape)
        index.add(embeddings.numpy())
        faiss_index_lookup_samples[index_id] = waveform
        index_id = index_id + 1

ap3087000.wav
./data/ap3087000.wav
torch.Size([1, 1214, 768])
torch.Size([1, 768])
hh29.wav
hh15.wav
hh9.wav
hh14.wav
hh8.wav
hh28.wav
ap441000.wav
./data/ap441000.wav
torch.Size([1, 1214, 768])
torch.Size([1, 768])
hh16.wav
ap1323000.wav
./data/ap1323000.wav
torch.Size([1, 1214, 768])
torch.Size([1, 768])
hh17.wav
ap3969000.wav
./data/ap3969000.wav
torch.Size([1, 1214, 768])
torch.Size([1, 768])
hh13.wav
.DS_Store
hh12.wav
ap4189500.wav
./data/ap4189500.wav
torch.Size([1, 1214, 768])
torch.Size([1, 768])
ap220500.wav
./data/ap220500.wav
torch.Size([1, 1214, 768])
torch.Size([1, 768])
hh10.wav
hh38.wav
ap1102500.wav
./data/ap1102500.wav
torch.Size([1, 1214, 768])
torch.Size([1, 768])
ap3528000.wav
./data/ap3528000.wav
torch.Size([1, 1214, 768])
torch.Size([1, 768])
hh39.wav
hh11.wav
ap882000.wav
./data/ap882000.wav
torch.Size([1, 1214, 768])
torch.Size([1, 768])
ap7497000.wav
./data/ap7497000.wav
torch.Size([1, 1214, 768])
torch.Size([1, 768])
ap3748500.wav
./data/ap3748500.wav
torch.S

In [288]:
sr = 16000

In [289]:
waveform, sample_rate = torchaudio.load('./data/ap0.wav')
waveform = waveform.squeeze().numpy()
display(Audio(waveform, rate=sr))

input_values = feature_extractor(waveform, return_tensors="pt", sampling_rate=sample_rate)
embeddings = model(**input_values).last_hidden_state
embeddings = embeddings.mean(dim=1)


In [290]:
D, I = index.search(embeddings.detach().numpy(), k=5)


In [291]:
print(D)

[[ 0.       41.831356 75.09376  89.35241  92.90102 ]]


In [292]:
print(I[0])

[20  5  1 15 17]


In [297]:
matched_wave_form = faiss_index_lookup_samples[I[0][2]]


In [298]:
display(Audio(matched_wave_form, rate=sample_rate))
print()